In [82]:
import json
import pandas as pd

In [83]:
# function to get shortened wiki articles

N_SENTENCES=3
def get_sentences(x):
    n = N_SENTENCES + 1
    sentences = x.split('. ')
    try:
        return '. '.join(sentences[:n])
    except:
        return '. '.join(sentences)
def count_sentenes(x):
    return len(x.split('. '))

In [84]:
# read pickles

wiki_en = pd.read_pickle('data/wiki_en_summaries.pkl')
wiki_fr = pd.read_pickle('data/wiki_fr_summaries.pkl')
bnf_summaries = pd.read_pickle('data/bnf_summaries.pkl')
named_subjects = pd.read_pickle('data/named_subjects.pkl')
age_gender = pd.read_json('age_gender_labeles.json',orient='records')

In [85]:
# get name-id df (exploded)

named_subjects = named_subjects[['id','title','names']].explode('names')
named_subjects = named_subjects.set_index('id')
named_subjects = named_subjects.rename(columns = {'names':'name'})

In [86]:
# get name-id df (nested in list)

id_img = named_subjects.drop('title', axis=1)
id_img = id_img.reset_index()
id_img = pd.DataFrame(id_img.groupby('name')['id'].apply(list))
id_img = id_img.rename(columns={'id':'id_list'})

In [87]:
# organize dataframes, shorten texts, rename columns

wiki_en['wiki_en_text'] = wiki_en.summary.apply(get_sentences)
wiki_fr['wiki_fr_text'] = wiki_fr.summary_fr.apply(get_sentences)

wiki_en = wiki_en[['name','wiki_en_text','url','rank_en']]
wiki_fr = wiki_fr[['name','wiki_fr_text','url_fr','rank_fr']]
bnf_summaries = bnf_summaries[['raw_name','bnf_note','bnf_link']]

bnf_summaries = bnf_summaries.rename(columns={'raw_name':'name'})
wiki_en = wiki_en.rename(columns={'url':'wiki_en_link'})
wiki_fr = wiki_fr.rename(columns={'url':'wiki_fr_link'})

In [88]:
# link name and id defined in age_gender df

named_age_gender = named_subjects.merge(age_gender[['age','gender','id']], on='id')

In [135]:
# merge everything

data_final = pd.merge(bnf_summaries,wiki_en,how='outer')
data_final = pd.merge(data_final,wiki_fr,how='outer')
data_final = pd.merge(data_final, named_age_gender, how='outer')
data_final = data_final.merge(id_img, on='name',how='outer')

In [136]:
# redefine rank <<<<<< THIS WILL CHANGE 


data_final.rank_en.fillna(2000, inplace=True)
data_final.rank_fr.fillna(2000, inplace=True)

data_final['rank_'] = data_final[['rank_en','rank_fr']].mean(axis=1)

In [138]:
data_final = data_final.sort_values(by='rank_')

In [139]:
data_final = data_final[['id','name','id_list', 'title','rank_','bnf_note', 'bnf_link', 'wiki_en_text', 'wiki_en_link',
        'wiki_fr_text', 'url_fr', 'age','gender']]

In [91]:
data_final.to_pickle('data/merged_dataframe.pkl')

In [141]:
data_final.head(20)

,id,name,id_list,title,rank_,bnf_note,bnf_link,wiki_en_text,wiki_en_link,wiki_fr_text,url_fr,age,gender
235,https://gallica.bnf.fr/ark:/12148/btv1b53050879q,"Bouguereau, William (1825-1905)",[https://gallica.bnf.fr/ark:/12148/btv1b530508...,Bouguereau,3.50,"Peintre, dessinateur, aquarelliste. - Décorate...",http://data.bnf.fr/ark:/12148/cb12758427r#about,William-Adolphe Bouguereau (French pronunciati...,https://en.wikipedia.org/wiki/William-Adolphe_...,L'œuvre peint de William Bouguereau (La Rochel...,https://fr.wikipedia.org/wiki/%C5%92uvre_peint...,43.553167,0.093347
521,https://gallica.bnf.fr/ark:/12148/btv1b53104547j,"Delacroix, Eugène (1798-1863)",[https://gallica.bnf.fr/ark:/12148/btv1b531045...,Eug Delacroix,6.00,"Peintre, aquarelliste, graveur et lithographe",http://data.bnf.fr/ark:/12148/cb118991616#about,Ferdinand Victor Eugène Delacroix ( DEL-ə-krwa...,https://en.wikipedia.org/wiki/Eug%C3%A8ne_Dela...,Eugène Delacroix est un peintre français né le...,https://fr.wikipedia.org/wiki/Eug%C3%A8ne_Dela...,45.458278,0.057673
451,https://gallica.bnf.fr/ark:/12148/btv1b53050548w,"Courbet, Gustave (1819-1877)",[https://gallica.bnf.fr/ark:/12148/btv1b530505...,"Courbet, peintre",7.00,"Peintre, sculpteur et graveur. - A été aussi l...",http://data.bnf.fr/ark:/12148/cb120519068#about,"Jean Désiré Gustave Courbet (UK: KOOR-bay, US...",https://en.wikipedia.org/wiki/Gustave_Courbet,"Gustave Courbet, né le 10 juin 1819 à Ornans e...",https://fr.wikipedia.org/wiki/Gustave_Courbet,40.302552,0.051735
1573,https://gallica.bnf.fr/ark:/12148/btv1b53187673w,"Robespierre, Maximilien de (1758-1794)",[https://gallica.bnf.fr/ark:/12148/btv1b531876...,M Robespierre Reproduction de tableau,8.00,Avocat. - Homme politique. - Révolutionnaire,http://data.bnf.fr/ark:/12148/cb11922216p#about,Maximilien François Marie Isidore de Robespier...,https://en.wikipedia.org/wiki/Maximilien_Robes...,"Maximilien de Robespierre, ou Maximilien Robes...",https://fr.wikipedia.org/wiki/Maximilien_de_Ro...,38.010203,0.479522
828,https://gallica.bnf.fr/ark:/12148/btv1b53066441r,"Grant, Ulysses Simpson (1822-1885)",[https://gallica.bnf.fr/ark:/12148/btv1b530664...,Général Grant,9.00,Général et homme d'État. - Président des États...,http://data.bnf.fr/ark:/12148/cb12157552z#about,Ulysses S. Grant (born Hiram Ulysses Grant; Ap...,https://en.wikipedia.org/wiki/Ulysses_S._Grant,"Ulysses S. Grant, né Hiram Ulysses Grant le 27...",https://fr.wikipedia.org/wiki/Ulysses_S._Grant,44.943712,0.019418
757,https://gallica.bnf.fr/ark:/12148/btv1b530658052,"Garibaldi, Giuseppe (1807-1882)",[https://gallica.bnf.fr/ark:/12148/btv1b530658...,Général Garibaldi,11.25,"Homme politique, héros du Risorgimento. - Insc...",http://data.bnf.fr/ark:/12148/cb11904146d#about,"Giuseppe Maria Garibaldi ( GARR-ib-AWL-dee, It...",https://en.wikipedia.org/wiki/Giuseppe_Garibaldi,Giuseppe (Jousé/Josep en niçois) Garibaldi (pr...,https://fr.wikipedia.org/wiki/Giuseppe_Garibaldi,40.449391,0.080095
596,https://gallica.bnf.fr/ark:/12148/btv1b53092261c,"Doré, Gustave (1832-1883)",[https://gallica.bnf.fr/ark:/12148/btv1b530796...,M Doré peintre,12.50,"Peintre, sculpteur, dessinateur et graveur. - ...",http://data.bnf.fr/ark:/12148/cb119004228#about,Paul Gustave Louis Christophe Doré (; French: ...,https://en.wikipedia.org/wiki/Gustave_Dor%C3%A9,Paul Gustave Louis Christophe Doré dit Gustave...,https://fr.wikipedia.org/wiki/Gustave_Dor%C3%A9,37.494515,0.324808
159,https://gallica.bnf.fr/ark:/12148/btv1b53162160w,"Bernhardt, Sarah (1844-1923)",[https://gallica.bnf.fr/ark:/12148/btv1b531621...,Sarah Bernhardt Jeanne d'Arc,15.00,Actrice. - Directrice de théâtre. - Auteur dra...,http://data.bnf.fr/ark:/12148/cb11891564d#about,Sarah Bernhardt (French: [saʁa bɛʁnɑʁt]; born ...,https://en.wikipedia.org/wiki/Sarah_Bernhardt,Sara Marie Henriette Bernhardt dite Sarah Bern...,https://fr.wikipedia.org/wiki/Sarah_Bernhardt,29.763913,0.471756
91,https://gallica.bnf.fr/ark:/12148/btv1b53118945v,"Balzac, Honoré de (1799-1850)",